In [1]:
import requests, json
from pprint import pprint
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'chips': [...],
  'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_config': {...},
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 11183396}


In [3]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])

{'assists': 0,
 'bonus': 0,
 'bps': 0,
 'can_select': False,
 'can_transact': True,
 'chance_of_playing_next_round': 0,
 'chance_of_playing_this_round': 0,
 'clean_sheets': 0,
 'clean_sheets_per_90': 0,
 'code': 438098,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -1,
 'cost_change_start_fall': 1,
 'creativity': '0.0',
 'creativity_rank': 713,
 'creativity_rank_type': 317,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 3,
 'ep_next': '0.0',
 'ep_this': '0.0',
 'event_points': 0,
 'expected_assists': '0.00',
 'expected_assists_per_90': 0,
 'expected_goal_involvements': '0.00',
 'expected_goal_involvements_per_90': 0,
 'expected_goals': '0.00',
 'expected_goals_conceded': '0.00',
 'expected_goals_conceded_per_90': 0,
 'expected_goals_per_90': 0,
 'first_name': 'Fábio',
 'form': '0.0',
 'form_rank': 704,
 'form_r

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Fábio Vieira,1,3
1,2,G.Jesus,1,4
2,3,Gabriel,1,2
3,4,Havertz,1,4
4,5,Hein,1,1


In [6]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1250,1360,1260,1370,1240,1350,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1110,1240,1080,1160,1140,1320,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1140,1175,1080,1140,1190,1210,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1080,1225,1070,1100,1090,1350,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1110,1150,1080,1100,1140,1200,131


In [7]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],75
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],247
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],324
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],80


In [8]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [9]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df


# show player #4's gameweek history
get_gameweek_history(392)[
    [
        'round',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,round,total_points,minutes,goals_scored,assists
0,1,1,20,0,0
1,2,5,32,0,1
2,3,9,60,1,0
3,4,8,45,1,0
4,5,10,90,1,0
5,6,2,76,0,0
6,7,3,61,0,0
7,8,1,25,0,0
8,9,2,67,0,0
9,10,1,5,0,0


In [10]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists
0,2022/23,40,500,1,2
1,2023/24,24,290,1,3


In [11]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,MID
1,7,Jorge Luiz,Frello Filho,Jorginho,Arsenal,MID
2,9,Gabriel,Martinelli Silva,Martinelli,Arsenal,MID
3,12,Ethan,Nwaneri,Nwaneri,Arsenal,MID
4,13,Martin,Ødegaard,Ødegaard,Arsenal,MID


In [12]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

  0%|          | 0/726 [00:00<?, ?it/s]

In [14]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(50)


,element,web_name,total_points,goals_scored,assists
327,328,M.Salah,211,18,13
181,182,Palmer,163,14,7
98,99,Mbeumo,138,13,3
400,401,Isak,136,15,5
350,351,Haaland,125,16,1
446,447,Wood,123,13,1
540,541,Cunha,112,10,4
16,17,Saka,107,5,11
57,58,Watkins,102,9,4
246,247,Iwobi,102,7,3
